In [30]:
// Source adapted from: https://dev.to/andreygermanov/how-to-create-yolov8-based-object-detection-web-service-using-python-julia-nodejs-javascript-go-and-rust-4o8e#process_the_output_nodejs
interface Config {
    modelPath: string;
    modelInputWidth: number;
    modelInputHeight: number;
    sourceImage: string;
    destinationImage: string;
    modelClassNames: string[];
}

const modelConfig:Config = {
    modelPath: './models/lpr-8n.onnx',
    modelInputWidth: 640,
    modelInputHeight: 640,
    sourceImage: './sample-images/car-with-licenseplate-resized.jpeg', //./sample-images/pass-me.jpg
    destinationImage: './sample-images/pass-me-detected2.png',
    modelClassNames: ['license plate']
};
console.log('configuration loaded');

configuration loaded


In [31]:
const sharp = require('sharp') as typeof import('sharp');

interface ImageData {
    input: number[];
    width: number;
    height: number;
}

const prepareImage = async (imagePath: string, modelInputWidth:number, modelInputHeight:number): Promise<ImageData> => {
    // load config.sourceImage from disk into sharp
    const image = sharp(imagePath);
    const imageMetadata = await image.metadata();
    const [width, height] = [imageMetadata.width, imageMetadata.height];
    if(!width || !height) throw new Error('Could not read image dimensions');
    const pixels = await image.removeAlpha() // remove alpha channel, model accepts RGB only
        .resize({width: modelInputWidth, height: modelInputHeight, fit: 'fill'}) // resize to fit model dimensions
        .raw()
        .toBuffer(); // convert to a single array of pixels

    const red:number[] = [], green:number[] = [], blue:number[] = [];
    for(let index = 0; index < pixels.length; index += 3) {
        red.push(pixels[index]/255.0); // normalize to [0, 1] by dividing by 255
        green.push(pixels[index + 1]/255.0); // normalize to [0, 1] by dividing by 255
        blue.push(pixels[index + 2]/255.0);// normalize to [0, 1] by dividing by 255
    }
    const input = [...red, ...green, ...blue] ; // concat RGB channels into a single array

    return {
        input, width, height};
}

const imageData:ImageData = await prepareImage(modelConfig.sourceImage, modelConfig.modelInputWidth, modelConfig.modelInputHeight);
console.log(`Image data prepared: ${imageData.input.length} elements`);

export {}

Image data prepared: 1228800 elements


In [32]:
const ort = require('onnxruntime-node') as typeof import('onnxruntime-node'); // onnyruntime-web

const runModel = async (modelPath:string,modelWidth:number, modelHeight:number, input:number[]): Promise<Float32Array> => {
  const session = await ort.InferenceSession.create(modelPath);
  const imageTensor = new ort.Tensor(Float32Array.from(input), [1, 3, modelHeight, modelWidth]);
  const outputs = await session.run({images: imageTensor});
  console.log(outputs);
  return outputs['output0'].data as Float32Array;
}

const modelOutput = await runModel(modelConfig.modelPath,modelConfig.modelInputWidth, modelConfig.modelInputHeight, imageData.input)
console.log(`Model run output: ${modelOutput.length} elements`)

export {}

{
  output0: h {
    dims: [ 1, 5, 8400 ],
    type: 'float32',
    data: Float32Array(42000) [
      11.964938163757324, 11.403425216674805, 11.809263229370117,
       20.26769256591797,  42.34498596191406,  59.78722381591797,
      53.838314056396484, 48.435482025146484,  45.66758728027344,
      45.500701904296875,  56.43748474121094,   68.0949478149414,
       115.5863037109375, 115.85722351074219, 123.11265563964844,
      124.60860443115234, 128.12933349609375, 137.31182861328125,
       142.8465118408203,  147.3687744140625,  149.6078338623047,
       160.6686553955078, 183.00067138671875, 196.51751708984375,
      197.77696228027344, 198.23965454101562, 200.84410095214844,
        200.899169921875,  201.9393768310547, 207.68527221679688,
      214.67669677734375, 221.81927490234375,  229.3299560546875,
      248.27902221679688, 267.56146240234375, 284.56939697265625,
       302.1208801269531, 308.80657958984375,  311.3423767089844,
       315.9258117675781,  321.5726318359375, 

In [33]:
function iou(box1,box2) {
    return intersection(box1,box2)/union(box1,box2);
}

function union(box1,box2) {
    const [box1_x1,box1_y1,box1_x2,box1_y2] = box1;
    const [box2_x1,box2_y1,box2_x2,box2_y2] = box2;
    const box1_area = (box1_x2-box1_x1)*(box1_y2-box1_y1)
    const box2_area = (box2_x2-box2_x1)*(box2_y2-box2_y1)
    return box1_area + box2_area - intersection(box1,box2)
}

function intersection(box1,box2) {
    const [box1_x1,box1_y1,box1_x2,box1_y2] = box1;
    const [box2_x1,box2_y1,box2_x2,box2_y2] = box2;
    const x1 = Math.max(box1_x1,box2_x1);
    const y1 = Math.max(box1_y1,box2_y1);
    const x2 = Math.min(box1_x2,box2_x2);
    const y2 = Math.min(box1_y2,box2_y2);
    return (x2-x1)*(y2-y1)
}

console.log('functions defined');

functions defined


In [34]:

function process_output(output, img_width, img_height,classNames: string[]) {
    //initialize the result array in the format of [x1,y1,x2,y2,label,prob] making it TypeSafe
    let boxes:[number,number,number,number,string,number][] = [];

    for (let index=0;index<8400;index++) {
        const [classId,prob] = ([...Array(80).keys()] as [number,number])
            .map(col => [col, output[8400*(col+4)+index]])
            .reduce((accum, item) => item[1]>accum[1] ? item : accum,[0,0]);
        if (prob  < 0.5) {
            continue;
        }
        const label = classNames[classId];
        const xc = output[index];
        const yc = output[8400+index];
        const w = output[2*8400+index];
        const h = output[3*8400+index];
        const x1 = Math.round((xc-w/2)/640*img_width);
        const y1 = Math.round((yc-h/2)/640*img_height);
        const x2 = Math.round((xc+w/2)/640*img_width);
        const y2 = Math.round((yc+h/2)/640*img_height);
        boxes.push([x1,y1,x2,y2,label,prob]);
    }
    console.log(`boxes - count: ${boxes.length}`);
    boxes = boxes.sort((box1,box2) => box2[5]-box1[5])
    const result:[number,number,number,number,string,number][] = [];
    while (boxes.length>0) {
        result.push(boxes[0]);
        boxes = boxes.filter(box => iou(boxes[0],box)<0.7);
    }
    return result;
}

const arrayResult = process_output(modelOutput, imageData.width, imageData.height,modelConfig.modelClassNames);
console.log(`arrayResult - count: ${arrayResult.length}`);
console.log(`arrayResult - first item: ${JSON.stringify(arrayResult[0])}`);


boxes - count: 10
arrayResult - count: 1
arrayResult - first item: [243,213,424,324,"license plate",0.8836046457290649]


In [35]:
const { JSDOM } = require('jsdom') as typeof import('jsdom');
const { Canvas, createCanvas, Image, ImageData, loadImage } = require('canvas') as typeof import('canvas');
const jo = require('jpeg-autorotate') as typeof import('jpeg-autorotate');
import fs from 'fs';

function installDOM() {
  const dom = new JSDOM();
  global.document = dom.window.document;
  // The rest enables DOM image and canvas and is provided by node-canvas
  global.Image = Image;
  global.HTMLCanvasElement = Canvas;
  global.ImageData = ImageData;
  global.HTMLImageElement = Image;
}

installDOM();

async function prepare_input(imgPath,boxes:[number, number, number, number, string, number][]) {
    let buffer = fs.readFileSync(imgPath);
    const img = await loadImage(buffer);
          
          const canvas = createCanvas(img.width, img.height);
          const ctx = canvas.getContext("2d");
          ctx.drawImage(img,0,0);
              ctx.strokeStyle = "#00FF00";
              ctx.lineWidth = 3;
              ctx.font = "18px serif";
              boxes.forEach(([x1,y1,x2,y2,label]) => {
                  ctx.strokeRect(x1,y1,x2-x1,y2-y1);
                  ctx.fillStyle = "#00ff00";
                  const width = ctx.measureText(label).width;
                  ctx.fillRect(x1,y1,width+10,25);
                  ctx.fillStyle = "#000000";
                  ctx.fillText(label, x1, y1+18);
              });
          //write canvas to file
          const out = fs.createWriteStream(modelConfig.destinationImage);
          const stream = canvas.createPNGStream();
          stream.pipe(out);
          out.on('finish', () =>  console.log('The PNG file was created.'));

}

await prepare_input(modelConfig.sourceImage,arrayResult)
console.log('DOM Initialized to allow canvas and image');

export {}

DOM Initialized to allow canvas and image


The PNG file was created.
